## GPT-OSS Models

The OpenAI GPT-OSS models do not work using the standard MLX prompting.  

To illustrate this, we will use a standard prompt using a Qwen model and then try with a GPT-OSS model.

In [1]:
from mlx_lm import load, generate
from utilities import get_model

model, tokenizer, _ = get_model("qwen")
response = generate(model, tokenizer, "What is the capital of the United States?", verbose=True)

Loading model: mlx-community/Qwen3-4B-Instruct-2507-4bit
(First time may take a while...)


Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

✓ Model loaded successfully!
The capital of the United States is Washington, D.C. (Washington, District of Columbia). It is located in the eastern part of the country and serves as the seat of the federal government, housing the U.S. Congress, the White House, and other key government institutions. 

Note: While Washington, D.C. is the capital, it is not one of the 50 U.S. states, but rather a federal district established in 1790. The city is named after George Washington, the first U.S. president. 

✅ Correct answer: **Washington, D.C.**.
Prompt: 9 tokens, 46.225 tokens-per-sec
Generation: 125 tokens, 83.959 tokens-per-sec
Peak memory: 2.321 GB


There is a fair amount of meandering in this response, and that is because we did not apply any kind of chat template.  Let's do that now with the standard MLX chat template.

In [2]:
user_query = "What is the capital of the United States?"
if tokenizer.chat_template is not None:
    messages = [{"role":"user", "content":user_query}]
    prompt = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=False,
    )
else:
    prompt = user_query

print(prompt)
response = generate(model, tokenizer, prompt, verbose=True)

<|im_start|>user
What is the capital of the United States?<|im_end|>
<|im_start|>assistant

The capital of the United States is Washington, D.C.
Prompt: 17 tokens, 118.853 tokens-per-sec
Generation: 13 tokens, 90.936 tokens-per-sec
Peak memory: 2.338 GB


Clearly that is much better.  You can see that the chat template is applied and the response is much more focused.

Now let's try the same with a GPT-OSS model.


In [3]:
model, tokenizer, _ = get_model("gpt")

user_query = "What is the capital of the United States?"
if tokenizer.chat_template is not None:
    messages = [{"role":"user", "content":user_query}]
    prompt = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=False,
    )
else:
    prompt = user_query

print(prompt)
response = generate(model, tokenizer, prompt, verbose=True)

Loading model: mlx-community/gpt-oss-20b-MXFP4-Q4
(First time may take a while...)


Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

✓ Model loaded successfully!
<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.
Knowledge cutoff: 2024-06
Current date: 2026-01-06

Reasoning: medium

# Valid channels: analysis, commentary, final. Channel must be included for every message.<|end|><|start|>user<|message|>What is the capital of the United States?<|end|><|start|>assistant
<|channel|>analysis<|message|>The user asks: "What is the capital of the United States?" They likely mean "United States" but likely "United States" is a typo for "United States" maybe "United States" meaning "United States" maybe "United States" meaning "United States" maybe "United States" meaning "United States" maybe "United States" meaning "United States" maybe "United States" meaning "United States" maybe "United States" meaning "United States" maybe "United States" meaning "United States" maybe "United States" meaning "United States" maybe "United States" meaning "United States" maybe "United States" meaning "Uni

We never get an answer.  The output looks confusing.  There is a system message, a list of "valid channels", and then all of the response is in the "analysis" channel.  We're not in Kansas anymore, Toto.  Maybe we need to let it generate more tokens and think longer.

In [4]:
# response = generate(model, tokenizer, prompt, verbose=True, max_tokens=2048)

That didn't solve our problem but it confirms that the model is in an infinite loop with output to the analysis channel, which represents the "thinking".  Remember that these models are reasoning (thinking) models.

The solution to our problem is to use the openai-harmony library instead of the default MLX library.  Let's see how that works. 





In [5]:
from openai_harmony import (
    HarmonyEncodingName,
    Message,
    load_harmony_encoding,
    Conversation,
    Message,
    Role,
    SystemContent,
    DeveloperContent,
)

encoding = load_harmony_encoding(HarmonyEncodingName.HARMONY_GPT_OSS)


You have probably had OPENAI API experience with a system prompt as well as the user input. The GPT-OSS models have a different structure, but to make adoption easier, what we have considered a system prompt becomes a "developer" message.  The user input is still a "user" message.

In [6]:
from pprint import pprint

messages = [Message.from_role_and_content(Role.SYSTEM, SystemContent.new())]
pprint(messages)

[Message(author=Author(role=<Role.SYSTEM: 'system'>, name=None), content=[SystemContent(model_identity='You are ChatGPT, a large language model trained by OpenAI.', reasoning_effort=<ReasoningEffort.MEDIUM: 'Medium'>, conversation_start_date=None, knowledge_cutoff='2024-06', channel_config=ChannelConfig(valid_channels=['analysis', 'commentary', 'final'], channel_required=True), tools=None)], channel=None, recipient=None, content_type=None)]


We have some useful utilities to help us understand the messages.

In [7]:
from utilities.harmony_tools import (
    print_harmony_messages,
    display_harmony_response,
    extract_channel_content,
    get_final_response
)


In [ ]:

print_harmony_messages(messages)


🔵 SYSTEM
   Identity: You are ChatGPT, a large language model trained by OpenAI.
   Reasoning: ReasoningEffort.MEDIUM
   Knowledge Cutoff: 2024-06
   Valid Channels: ['analysis', 'commentary', 'final']



In [9]:
system_message = "You are a friendly assistant and you always answer like a Pirate."
user_message = "What is the capital of the United States?"

if system_message:
    messages.append(
        Message.from_role_and_content(
            Role.DEVELOPER,
            DeveloperContent.new().with_instructions(system_message)
        )
    )

messages.append(Message.from_role_and_content(Role.USER, user_message))

print_harmony_messages(messages)



🔵 SYSTEM
   Identity: You are ChatGPT, a large language model trained by OpenAI.
   Reasoning: ReasoningEffort.MEDIUM
   Knowledge Cutoff: 2024-06
   Valid Channels: ['analysis', 'commentary', 'final']


🔵 DEVELOPER
   Instructions: You are a friendly assistant and you always answer like a Pirate.


🔵 USER
   Text: What is the capital of the United States?



Now we create a conversation object, do some fancy encoding and generate a prompt.

In [10]:
 # Create conversation
convo = Conversation.from_messages(messages)
        
# Render for completion and decode to string
prefill_ids = encoding.render_conversation_for_completion(convo, Role.ASSISTANT)
prompt = encoding.decode(prefill_ids)

print(prompt)

<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.
Knowledge cutoff: 2024-06

Reasoning: medium

# Valid channels: analysis, commentary, final. Channel must be included for every message.<|end|><|start|>developer<|message|># Instructions

You are a friendly assistant and you always answer like a Pirate.<|end|><|start|>user<|message|>What is the capital of the United States?<|end|><|start|>assistant


In [11]:
response = generate(model, tokenizer, prompt, verbose=True)

<|channel|>analysis<|message|>The user asks: "What is the capital of the United States?" The instructions: The system says: "You are ChatGPT, a large language model trained by OpenAI." The developer says: "You are a friendly assistant and you always answer like a pirate." So we must answer in a pirate style. The user question: "What is the capital of the United States?" The answer: Washington, D.C. We should respond in a pirate style. So we should say something like "Arrr, the capital be Washington, D.C." Or something like that. We must comply with policy. There's no disallowed content. It's a normal question. So we can comply. We should answer in a pirate style. So we can say: "Ahoy! The capital of the United States be Washington, D.C." Or something. Let's do that.<|end|><|start|>assistant<|channel|>final<|message|>Ahoy, matey! The grand capital o’ the United States be Washington, D.C., the mighty seat o’ the government, where the flag waves high and the eagle soars proud. Arr!
Prompt

This shows all the output but it would be nice to see the output broken down by channels.  Let's do that.

In [14]:
display_harmony_response(response)


HARMONY RESPONSE BREAKDOWN

────────────────────────────────────────────────────────────────────────────────
📍 CHANNEL: ANALYSIS
────────────────────────────────────────────────────────────────────────────────
The user asks: "What is the capital of the United States?" The instructions: The system says: "You are ChatGPT, a large language model trained by OpenAI." The developer says: "You are a friendly assistant and you always answer like a pirate." So we must answer in a pirate style. The user question: "What is the capital of the United States?" The answer: Washington, D.C. We should respond in a pirate style. So we should say something like "Arrr, the capital be Washington, D.C." Or something like that. We must comply with policy. There's no disallowed content. It's a normal question. So we can comply. We should answer in a pirate style. So we can say: "Ahoy! The capital of the United States be Washington, D.C." Or something. Let's do that.

─────────────────────────────────────────

OPENAI stresses the importance of handling the channels correctly.  We only have two channels in this response, but ONLY the "final" channel has the answer.  This is the only channel that should be shown to the user.


In [16]:
get_final_response(response)

'Ahoy, matey! The grand capital o’ the United States be Washington, D.C., the mighty seat o’ the government, where the flag waves high and the eagle soars proud. Arr!'


The analysis channel has not been safety trained to the same degree as the final channel.  It is possible that the analysis channel will generate content that is not safe for the user.  Therefore, we need to be careful about what we show to the user.

It would be nice to be able to extract the channel contents so that we can understand how our applications work.  Let's illustrate that.

In [ ]:

analysis = extract_channel_content(response, 'analysis')
commentary = extract_channel_content(response, 'commentary')
final = extract_channel_content(response, 'final')

print("Analysis:", analysis)
print("\nCommentary:", commentary)
print("\nFinal:", final)

Analysis: The user asks: "What is the capital of the United States?" The instructions: The system says: "You are ChatGPT, a large language model trained by OpenAI." The developer says: "You are a friendly assistant and you always answer like a pirate." So we must answer in a pirate style. The user question: "What is the capital of the United States?" The answer: Washington, D.C. We should respond in a pirate style. So we should say something like "Arrr, the capital be Washington, D.C." Or something like that. We must comply with policy. There's no disallowed content. It's a normal question. So we can comply. We should answer in a pirate style. So we can say: "Ahoy! The capital of the United States be Washington, D.C." Or something. Let's do that.

Commentary: None

Final: Ahoy, matey! The grand capital o’ the United States be Washington, D.C., the mighty seat o’ the government, where the flag waves high and the eagle soars proud. Arr!


We have created a sophisticated generate_response function that will decide if a model is a GPT-OSS model and will format our prompt correctly.  This is in the utilities.utils.py file.  Let's try it.

In [17]:
from utilities.utils import generate_response, generate_response_with_system

In [20]:
prompt = "WHat is the capital of the United States?"
response = generate_response(model, tokenizer, prompt,max_tokens=2048)
get_final_response(response)

User message: WHat is the capital of the United States?

<|channel|>analysis<|message|>The user asks: "WHat is the capital of the United States?" They want the capital of the United States. The answer: Washington, D.C. The user typed "United States" but likely means "United States" or "United States" maybe "United States" but they typed "United States". The correct answer: Washington, D.C. The user typed "WHat" with capital W and H. They might want the answer. There's no policy violation. It's a straightforward question. We can answer. The user typed "WHat" maybe they want the capital. There's no disallowed content. So we can comply. The answer: Washington, D.C.<|end|><|start|>assistant<|channel|>final<|message|>The capital of the United States is Washington, D.C.



'The capital of the United States is Washington, D.C.'

In [21]:
display_harmony_response(response)


HARMONY RESPONSE BREAKDOWN

────────────────────────────────────────────────────────────────────────────────
📍 CHANNEL: ANALYSIS
────────────────────────────────────────────────────────────────────────────────
The user asks: "WHat is the capital of the United States?" They want the capital of the United States. The answer: Washington, D.C. The user typed "United States" but likely means "United States" or "United States" maybe "United States" but they typed "United States". The correct answer: Washington, D.C. The user typed "WHat" with capital W and H. They might want the answer. There's no policy violation. It's a straightforward question. We can answer. The user typed "WHat" maybe they want the capital. There's no disallowed content. So we can comply. The answer: Washington, D.C.

────────────────────────────────────────────────────────────────────────────────
📍 CHANNEL: FINAL
────────────────────────────────────────────────────────────────────────────────
The capital of the United

In [24]:
response = generate_response_with_system(model, tokenizer, prompt, system_message)
get_final_response(response)

User message: WHat is the capital of the United States?

System: You are a friendly assistant and you always answer like a Pirate.

<|channel|>analysis<|message|>We have a conversation. The user asks: "WHat is the capital of the United States?" The user typed "WHat" with a capital W and a lowercase h? Actually "WHat" is "WHat" with capital W, H, a, t? Wait: "WHat" is "WHat" with capital W, H, a, t? Actually "WHat" is "WHat" with capital W, H, a, t? The user typed "WHat" with capital W, H, a, t? The user typed "WHat" with capital W, H, a, t? The user typed "WHat" with capital W, H, a, t? The user typed "WHat" with capital W, H, a, t? The user typed "WHat" with capital W, H, a, t? The user typed "WHat" with capital W, H, a, t? The user typed "WHat" with capital W, H, a, t? The user typed "WHat" with capital W, H, a, t? The user typed "WHat" with capital W, H, a, t? The user typed



'[Model did not complete response - try increasing max_tokens]'